# Word Embeddings : le modèle Word2Vec

## Imports

In [1]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [2]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [3]:
infile = f"../data/sents.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [4]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [5]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [6]:
len(bigram_phrases.vocab.keys())

15850647

Prenons une clé au hasard :

In [9]:
key_ = list(bigram_phrases.vocab.keys())[156]
print(key_)

410


Le dictionnaire indique le score de cette coocurrence :

In [10]:
bigram_phrases.vocab[key_]

3508

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [11]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [13]:
trigram_phrases = Phrases(bigram_phraser[sentences])

MemoryError: 

In [13]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [14]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [15]:
print(corpus[:100])

[['mi', 'imnri', 'r', 'i', '<<', 'i', 'i', 'hmu', "'", 'i', '/', 'tx', "-'", 'l', ':', 'marche', 'tenu', 'hors', 'villa', ',', 'la', '9', '.'], ['--', 'u', 'a', 'ete', 'vaain', 'si', 'teicj', '>>', 'm', 'races', 'indigenes', 'de', 'fr', '.'], ['31', '<)', 'a', '5s', "'", 'k', '131', 'de', '.'], ['rasa', 'iichakdui', "'", 'te', ',', 'do', '(', 'r', '.', '3s0', 'h', '710', '.', 'taureaux', 'iallsenas', ',>', 'ia', 'u', '\\', '--', 'a', '--', ';', '0ii', '.'], ['hollandais', ',', 'dufr', '.'], ['0', '.'], ['--', 'a', '9', '.--', 'la', 'idto', '-', 'vachei', 'laitieres', ':', 'bn', 'vante', '1q', '.'], ['vendues', '3', '\\', 'au', 'prix', 'la', '410', 'a', '*', '<<', 'i', 'h', '\\;', 'genisses', ',', 'kl', '.'], ["'.", '9', '.'], ['i', 'l', '.', '2', 'i', '.', 'id', '.'], ['da', '370', 'i', '6lutr', '.'], ['marche', 'a', '<', 'u', 'porcs', '.'], ['--', 'categorie', 'de', 'lt', 'ilashtya', ':', "'", '237', 'on', 'vente', ';', 'vendus', '1', 'm', '.', 'do', "'", '2', 'i', '.--', 'a', ';:,', 

## Entrainement d'un modèle Word2Vec sur ce corpus

In [16]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: total: 9.48 s
Wall time: 29.2 s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [17]:
outfile = f"../data/newspapers.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [18]:
model = Word2Vec.load("../data/newspapers.model")

### Imprimer le vecteur d'un terme

In [19]:
model.wv["ministre"]

array([ 1.1970322 , -0.07318376,  0.6614855 ,  0.50884306,  0.9949899 ,
        1.1116452 ,  1.1065866 , -1.0641718 , -1.1435872 ,  0.35503915,
       -0.08110788,  4.413815  ,  2.025264  , -0.8961699 , -2.1539612 ,
       -2.8553195 , -0.5082356 , -1.6796227 , -5.000319  ,  2.0318856 ,
        3.0283687 ,  2.21347   ,  0.97804   , -2.4381998 , -4.860589  ,
        1.9145362 ,  0.98774076, -0.4126336 , -2.4837644 ,  0.21860567,
        0.01903898, -0.19279872], dtype=float32)

### Calculer la similarité entre deux termes

In [20]:
model.wv.similarity("ministre", "roi")

0.5841106

### Chercher les mots les plus proches d'un terme donné

In [21]:
model.wv.most_similar("ministre", topn=10)

[('secretaire', 0.9039165377616882),
 ('membre', 0.8936543464660645),
 ('president', 0.8816959261894226),
 ('inspecteur', 0.8704195618629456),
 ('adjoint', 0.8627258539199829),
 ('conseiller', 0.8571282029151917),
 ('depute', 0.8563404083251953),
 ('professeur', 0.8490974307060242),
 ('vice_-_president', 0.8458018898963928),
 ('directeur', 0.8434743881225586)]

### Faire des recherches complexes à travers l'espace vectoriel

In [27]:
print(model.wv.most_similar(positive=['paris','belgique'], negative=['france']))

[('londres', 0.7874029874801636), ('ostende', 0.7817729115486145), ('amsterdam', 0.781009316444397), ('new_-_york', 0.7674489617347717), ('liege', 0.7646644115447998), ('geneve', 0.7633039951324463), ('hambourg', 0.7402405738830566), ('grand_festival', 0.7396638989448547), ('1893', 0.7329071164131165), ('madrid', 0.7319207787513733)]
